<a href="https://colab.research.google.com/github/et-tu-atulu/dog_breed_identification/blob/master/VG19_pup_breed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#extract the zip file here
import os
from zipfile import ZipFile
zipobj=ZipFile(os.path.abspath('.')+'/drive/My Drive/dataset.zip', 'r')
zipobj.extractall()

In [3]:
!pip install tf-nightly-gpu

     |████████████████████████████████| 397.0MB 45kB/s 
     |████████████████████████████████| 61kB 21.9MB/s 
     |████████████████████████████████| 3.2MB 29.6MB/s 
     |████████████████████████████████| 501kB 40.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/51/3e/a3/b351fae0cbf15373c2136a54a70f43fea5fe91d8168a5faaa4
Successfully built opt-einsum


In [3]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator


image_gen_train = ImageDataGenerator(rescale = 1./255, horizontal_flip = True, zoom_range = 0.2, rotation_range=90)

train_gen = image_gen_train.flow_from_directory(batch_size=32, directory = os.path.abspath('.')+'/train', shuffle = True, target_size =(150, 150), class_mode = 'sparse')

image_gen_test = ImageDataGenerator(rescale = 1./255)

test_gen = image_gen_test.flow_from_directory(batch_size=32, directory=os.path.abspath('.')+'/test', target_size=(150, 150), class_mode='sparse')



Found 16418 images belonging to 120 classes.
Found 4162 images belonging to 120 classes.


# New Section

In [4]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

#

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(120, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

    
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])


model.summary()

W0704 16:27:19.894055 140454757267328 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0704 16:27:19.936750 140454757267328 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0704 16:27:21.487849 140454757267328 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [5]:


EPOCHS = 50
history = model.fit_generator(train_gen,
                    steps_per_epoch=513,         
                    epochs=EPOCHS,
                    validation_data=test_gen,
                    validation_steps=130)

W0704 16:27:32.077281 140454757267328 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1251: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
247/513 [=============>................] - ETA: 1:24 - loss: 5.0112 - acc: 0.0502

KeyboardInterrupt: ignored